In [1]:
import pandas as pd
import zipfile
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

<h2>Criação dos dataframes</h2>

In [29]:
base_modificada = pd.read_excel('Database/Base Dados.xlsx', engine='openpyxl', sheet_name='Drivers')
base_construtores = pd.read_excel('Database/Base Dados.xlsx', engine='openpyxl', sheet_name='Constructors')

In [31]:
with zipfile.ZipFile('database_kaggle/archive.zip') as z:
    print(*z.namelist(), sep='\n')
    with z.open('drivers.csv') as f:
        df_drivers = pd.read_csv(f, encoding='ISO-8859-1')
    with z.open('constructors.csv') as c:
        df_orig_constructors = pd.read_csv(c, encoding='ISO-8859-1')

circuits.csv
constructorResults.csv
constructorStandings.csv
constructors.csv
driverStandings.csv
drivers.csv
lapTimes.csv
pitStops.csv
qualifying.csv
races.csv
results.csv
seasons.csv
status.csv


In [4]:
df_drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [5]:
base_modificada.head()

,driverId,forename,surname,nationality,picture_url
0,852,Yuki,Tsunoda,Japanese,https://www.formula1.com/content/dam/fom-websi...
1,822,Valtteri,Bottas,Finnish,https://www.formula1.com/content/dam/fom-websi...
2,815,Sergio,Pérez,Mexican,https://www.formula1.com/content/dam/fom-websi...
3,20,Sebastian,Vettel,German,https://www.formula1.com/content/dam/fom-websi...
4,842,Pierre,Gasly,French,https://www.formula1.com/content/dam/fom-websi...


<h2>id dos pilotos com url não preenchidos</h2>

In [6]:
base_modificada_url_nan = base_modificada[base_modificada['picture_url'].isna()]['driverId']

In [7]:
base_modificada_url_nan.head()

21    121
22    158
23    193
24    207
25    214
Name: driverId, dtype: int64

In [8]:
lista_id = base_modificada_url_nan.tolist()

In [9]:
lista_id

[121,
 158,
 193,
 207,
 214,
 239,
 240,
 249,
 288,
 309,
 332,
 342,
 344,
 353,
 364,
 366,
 381,
 386,
 392,
 393,
 396,
 403,
 405,
 418,
 433,
 434,
 438,
 440,
 449,
 451,
 452,
 463,
 468,
 470,
 494,
 501,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 545,
 548,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 572,
 574,
 575,
 576,
 584,
 591,
 592,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 611,
 612,
 613,
 614,
 615,
 616,
 618,
 628,
 629,
 630,
 631,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 667,
 673,
 674,
 675,
 676,
 677,
 678,
 679,
 680,
 693,
 699,
 700,
 701,
 702,
 726,
 727,
 728,
 729,
 730,
 731,
 732,
 733,
 734,
 737,
 766,
 767,
 768,
 769,
 770,
 771,
 772,
 794,
 795,
 796,
 797,
 798,
 799,
 803,
 804,
 805,
 51,
 56,
 65,
 66,
 71,
 84,
 87,
 95,
 118,
 135,
 141,
 151,
 156,


In [10]:
def get_driver_id(id: int) -> dict:
    driver_id = df_drivers.loc[df_drivers['driverId'] == id, ['driverId']].values[0][0]
    forename = df_drivers.loc[df_drivers['driverId'] == id, ['forename']].values[0][0]
    surname = df_drivers.loc[df_drivers['driverId'] == id, ['surname']].values[0][0]
    url = df_drivers.loc[df_drivers['driverId'] == id, ['url']].values[0][0]
    dict_info = {
        'driver_id': driver_id,
        'forename': forename,
        'surname': surname,
        'url': url
    }
    return dict_info

In [11]:
def dados_pilotos(lista_id_pilotos: list) -> list:
    lista_parametros = []
    for id in lista_id_pilotos:
        try:
            driver_id = df_drivers.loc[df_drivers['driverId'] == id, ['driverId']].values[0][0]
            forename = df_drivers.loc[df_drivers['driverId'] == id, ['forename']].values[0][0]
            surname = df_drivers.loc[df_drivers['driverId'] == id, ['surname']].values[0][0]
            url = df_drivers.loc[df_drivers['driverId'] == id, ['url']].values[0][0]
            dict_info = {
                'driver_id': driver_id,
                'forename': forename,
                'surname': surname,
                'url': url
            }
            lista_parametros.append(dict_info)
        except:
            continue

    return lista_parametros


In [12]:
get_driver_id(123)['url']

'http://en.wikipedia.org/wiki/Thierry_Boutsen'

In [106]:
dados_pilotos_tratados = dados_pilotos(lista_id)

In [107]:
dados_pilotos_tratados

[{'driver_id': 121,
  'forename': 'Michael',
  'surname': 'Andretti',
  'url': 'http://en.wikipedia.org/wiki/Michael_Andretti'},
 {'driver_id': 158,
  'forename': 'Eddie',
  'surname': 'Cheever',
  'url': 'http://en.wikipedia.org/wiki/Eddie_Cheever'},
 {'driver_id': 193,
  'forename': 'Danny',
  'surname': 'Sullivan',
  'url': 'http://en.wikipedia.org/wiki/Danny_Sullivan'},
 {'driver_id': 207,
  'forename': 'Mario',
  'surname': 'Andretti',
  'url': 'http://en.wikipedia.org/wiki/Mario_Andretti'},
 {'driver_id': 214,
  'forename': 'Kevin',
  'surname': 'Cogan',
  'url': 'http://en.wikipedia.org/wiki/Kevin_Cogan'},
 {'driver_id': 239,
  'forename': 'Brett',
  'surname': 'Lunger',
  'url': 'http://en.wikipedia.org/wiki/Brett_Lunger'},
 {'driver_id': 240,
  'forename': 'Danny',
  'surname': 'Ongais',
  'url': 'http://en.wikipedia.org/wiki/Danny_Ongais'},
 {'driver_id': 249,
  'forename': 'Bobby',
  'surname': 'Rahal',
  'url': 'http://en.wikipedia.org/wiki/Bobby_Rahal'},
 {'driver_id': 288

In [15]:
for piloto in dados_pilotos_tratados:
    print(piloto['driver_id'])
    print(piloto['url'])
    print()

121
http://en.wikipedia.org/wiki/Michael_Andretti

158
http://en.wikipedia.org/wiki/Eddie_Cheever

193
http://en.wikipedia.org/wiki/Danny_Sullivan

207
http://en.wikipedia.org/wiki/Mario_Andretti

214
http://en.wikipedia.org/wiki/Kevin_Cogan

239
http://en.wikipedia.org/wiki/Brett_Lunger

240
http://en.wikipedia.org/wiki/Danny_Ongais

249
http://en.wikipedia.org/wiki/Bobby_Rahal

288
http://en.wikipedia.org/wiki/Mark_Donohue

309
http://en.wikipedia.org/wiki/Peter_Revson

332
http://en.wikipedia.org/wiki/George_Follmer

342
http://en.wikipedia.org/wiki/Skip_Barber

344
http://en.wikipedia.org/wiki/Sam_Posey

353
http://en.wikipedia.org/wiki/Pete_Lovely

364
http://en.wikipedia.org/wiki/Dan_Gurney

366
http://en.wikipedia.org/wiki/Gus_Hutchison

381
http://en.wikipedia.org/wiki/Bobby_Unser

386
http://en.wikipedia.org/wiki/Richie_Ginther

392
http://en.wikipedia.org/wiki/Mike_Fisher_(driver)

393
http://en.wikipedia.org/wiki/Tom_Jones_(auto_racer)

396
http://en.wikipedia.org/wiki/Bob_B

<h2>Extraindo imagem dos pilotos</h2>

<h3>Imagem de um piloto como teste</h3>

In [16]:
def get_url_imagem_id(id: int, url: str) -> tuple:
    """

    :param id id do piloto :
    :param url url do piloto:
    :return tupla com id, url e imagem do piloto:
    """
    browsers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    page = requests.get(url, headers=browsers)
    respostas = page.text
    soup = BeautifulSoup(respostas, 'html.parser')
    # piloto = soup.find('td', class_= {'infobox-image'}).find('a').find('img')
    piloto = soup.find('td', class_={'infobox-image'}).find('img').get('src')

    return id, piloto

In [17]:
get_url_imagem_id(123, 'http://en.wikipedia.org/wiki/Thierry_Boutsen')

(123,
 '//upload.wikimedia.org/wikipedia/commons/thumb/2/20/Thierry_Boutsen_1993.jpg/220px-Thierry_Boutsen_1993.jpg')

In [18]:
def get_soup(url: str):
    browsers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    page = requests.get(url, headers=browsers)
    respostas = page.text
    soup = BeautifulSoup(respostas, 'html.parser')
    return soup


In [19]:
get_soup('https://www.statsf1.com/pt/kevin-cogan.aspx')


<!DOCTYPE html>

<html id="ctl00_html" lang="pt">
<head><title>
	Kevin COGAN • STATS F1
</title><meta charset="utf-8"/><meta content="Kevin COGAN" name="keywords"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Sport" name="category"/><meta content="STATS F1" name="author"/><meta content="#176AC7" name="theme-color"/><link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/include/jquery/jquery-ui.min.css" rel="stylesheet" type="text/css"/><link href="/include/master-c.css" media="screen and (min-width: 1024px)" rel="stylesheet" type="text/css"/><link href="/include/mobile-c.css" media="screen and (max-width: 1023px)" rel="stylesheet" type="text/css"/>
<script src="/include/master-c.js" type="text/javascript"></script>
<link href="/include/scrollable.css" rel="stylesheet" type="text/css"/>
<script async="" src="/include/pilote.js" type="text/javascript"></s

<h3>Imagens do site</h3>

<h3>imagens de todos os pilotos</h3>

In [21]:
def obtem_img_url_todos_pilotos(lista_pilotos: list) -> dict:
    """

    :param lista_pilotos lista de pilotos com id e url:
    :return: dicionario tratado com id e urk da imagem
    """
    pilotos_sem_imagens = 0
    lista_piloto_url = []
    browsers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    for piloto in tqdm(lista_pilotos):

        try:
            page = requests.get(piloto['url'], headers=browsers)
            respostas = page.text
            soup = BeautifulSoup(respostas, 'html.parser')
            url_piloto_img = soup.find('td', class_={'infobox-image'}).find('img').get('src')

            dict_piloto_url = {
                'driver_id': piloto['driver_id'],
                'url_imagem': url_piloto_img
            }
            lista_piloto_url.append(dict_piloto_url)
        except:
            pilotos_sem_imagens += 1
            with open('Database/url_img_faltantes.txt', 'a') as arq:
                arq.write(f"{piloto['url']} \n")

            continue

    print(f'pilotos sem imagens {pilotos_sem_imagens}')

    return lista_piloto_url

In [22]:
lista_imagens = obtem_img_url_todos_pilotos(dados_pilotos_tratados)

100%|██████████| 828/828 [14:17<00:00,  1.04s/it]

pilotos sem imagens 390


In [23]:
lista_imagens

[{'driver_id': 121,
  'url_imagem': '//upload.wikimedia.org/wikipedia/commons/thumb/7/71/Michael_Andretti_2007_Michigan.jpg/220px-Michael_Andretti_2007_Michigan.jpg'},
 {'driver_id': 158,
  'url_imagem': '//upload.wikimedia.org/wikipedia/commons/thumb/8/87/Eddie_Cheever_Jr_2009_Indy_500_Second_Qual_Day.JPG/220px-Eddie_Cheever_Jr_2009_Indy_500_Second_Qual_Day.JPG'},
 {'driver_id': 193,
  'url_imagem': '//upload.wikimedia.org/wikipedia/commons/thumb/1/18/DannySullivanAt2015indy500.JPG/220px-DannySullivanAt2015indy500.JPG'},
 {'driver_id': 207,
  'url_imagem': '//upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Mario_Andretti_Goodwood_Festival_of_Speed_2021_%28cropped%29.jpg/220px-Mario_Andretti_Goodwood_Festival_of_Speed_2021_%28cropped%29.jpg'},
 {'driver_id': 240,
  'url_imagem': '//upload.wikimedia.org/wikipedia/commons/thumb/f/f5/Ongais%2C_Danny_%281984%29.jpg/150px-Ongais%2C_Danny_%281984%29.jpg'},
 {'driver_id': 249,
  'url_imagem': '//upload.wikimedia.org/wikipedia/commons/thumb/

In [24]:
df_img = pd.DataFrame(lista_imagens)

In [26]:
# df_img.to_csv('Database/imagens_faltantes.csv')

In [25]:
df_img

,driver_id,url_imagem
0,121,//upload.wikimedia.org/wikipedia/commons/thumb...
1,158,//upload.wikimedia.org/wikipedia/commons/thumb...
2,193,//upload.wikimedia.org/wikipedia/commons/thumb...
3,207,//upload.wikimedia.org/wikipedia/commons/thumb...
4,240,//upload.wikimedia.org/wikipedia/commons/thumb...
...,...,...
433,835,//upload.wikimedia.org/wikipedia/commons/thumb...
434,836,//upload.wikimedia.org/wikipedia/commons/thumb...
435,837,//upload.wikimedia.org/wikipedia/commons/thumb...
436,838,//upload.wikimedia.org/wikipedia/commons/thumb...


<h2>Construtores Sem imagens</p>

In [32]:
base_construtores.head()

,constructorId,name,nationality,car_url
0,3,Williams,British,https://www.formula1.com/content/dam/fom-websi...
1,9,Red Bull,Austrian,https://www.formula1.com/content/dam/fom-websi...
2,131,Mercedes,German,https://www.formula1.com/content/dam/fom-websi...
3,1,McLaren,British,https://www.formula1.com/content/dam/fom-websi...
4,210,Haas F1 Team,American,https://www.formula1.com/content/dam/fom-websi...


In [37]:
df_orig_constructors

,constructorId,constructorRef,name,nationality,url,Unnamed: 5
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,NaN
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,NaN
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_F1,NaN
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,NaN
...,...,...,...,...,...,...
203,206,marussia,Marussia,Russian,http://en.wikipedia.org/wiki/Marussia_F1,NaN
204,207,caterham,Caterham,Malaysian,http://en.wikipedia.org/wiki/Caterham_F1,NaN
205,208,lotus_f1,Lotus F1,British,http://en.wikipedia.org/wiki/Lotus_F1,NaN
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport,NaN


In [40]:
df_contruct_sem_imagens = base_construtores[base_construtores['car_url'].isna()]

In [41]:
df_contruct_sem_imagens

,constructorId,name,nationality,car_url
10,16,BAR,British,NaN
11,19,Jaguar,British,NaN
12,21,Arrows,British,NaN
13,23,Brawn,British,NaN
14,24,Stewart,British,NaN
...,...,...,...,...
206,185,LDS-Alfa Romeo,South African,NaN
207,189,Eagle-Climax,American,NaN
208,190,Eagle-Weslake,American,NaN
209,205,Lotus,Malaysian,NaN


In [116]:
ids_contru_sem_img = base_construtores[base_construtores['car_url'].isna()]['constructorId'].tolist()

In [130]:
df_orig_constructors.loc[df_orig_constructors['constructorId'] == 185]

,constructorId,constructorRef,name,nationality,url,Unnamed: 5
182,185,lds-alfa_romeo,LDS-Alfa Romeo,South African,http://en.wikipedia.org/wiki/LDS_(automobile),NaN


In [136]:
def dados_construtores(lista_id_constr: list) -> list:
    lista_parametros = []

    for id in lista_id_constr:
        try:
            # print(df_orig_constructors.loc[df_orig_constructors['constructorId'] == id, ['constructorId']].values[0][0])
            # print(df_orig_constructors.loc[df_orig_constructors['constructorId'] == id, ['name']].values[0][0])
            # print(df_orig_constructors.loc[df_orig_constructors['constructorId'] == id, ['url']].values[0][0])
            dict_info = {
                'constructorId': df_orig_constructors.loc[df_orig_constructors['constructorId'] == id, ['constructorId']].values[0][0],
                'name': df_orig_constructors.loc[df_orig_constructors['constructorId'] == id, ['name']].values[0][0],
                'url': df_orig_constructors.loc[df_orig_constructors['constructorId'] == id, ['url']].values[0][0]
            }
            # print(dict_info['constructorId'])


        except:
            continue

        lista_parametros.append(dict_info)

    return lista_parametros

In [137]:
dados_construtores_tratados = dados_construtores(ids_contru_sem_img)

In [138]:
dados_construtores_tratados

[{'constructorId': 16,
  'name': 'BAR',
  'url': 'http://en.wikipedia.org/wiki/British_American_Racing'},
 {'constructorId': 19,
  'name': 'Jaguar',
  'url': 'http://en.wikipedia.org/wiki/Jaguar_Racing'},
 {'constructorId': 21,
  'name': 'Arrows',
  'url': 'http://en.wikipedia.org/wiki/Arrows_Grand_Prix_International'},
 {'constructorId': 23,
  'name': 'Brawn',
  'url': 'http://en.wikipedia.org/wiki/Brawn_GP'},
 {'constructorId': 24,
  'name': 'Stewart',
  'url': 'http://en.wikipedia.org/wiki/Stewart_Grand_Prix'},
 {'constructorId': 25,
  'name': 'Tyrrell',
  'url': 'http://en.wikipedia.org/wiki/Tyrrell_Racing'},
 {'constructorId': 26,
  'name': 'Lola',
  'url': 'http://en.wikipedia.org/wiki/MasterCard_Lola'},
 {'constructorId': 29,
  'name': 'Footwork',
  'url': 'http://en.wikipedia.org/wiki/Footwork_Arrows'},
 {'constructorId': 30,
  'name': 'Pacific',
  'url': 'http://en.wikipedia.org/wiki/Pacific_Racing'},
 {'constructorId': 31,
  'name': 'Simtek',
  'url': 'http://en.wikipedia.org

<h3> Construtores sem imagem</h3>

In [46]:
def obtem_img_url_todos_construtores(lista_construtores: list) -> dict:
    """

    :param lista_pilotos lista de pilotos com id e url:
    :return: dicionario tratado com id e urk da imagem
    """
    construtores_sem_imagens = 0
    lista_construtores_url = []
    browsers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    for construtor in tqdm(lista_construtores):

        try:
            page = requests.get(construtor['url'], headers=browsers)
            respostas = page.text
            soup = BeautifulSoup(respostas, 'html.parser')
            url_construtor_img = soup.find('td', class_={'infobox-image'}).find('img').get('src')

            dict_piloto_url = {
                'constructorId': construtor['constructorId'],
                'url_imagem': url_construtor_img
            }
            lista_construtores_url.append(dict_piloto_url)
        except:
            construtores_sem_imagens += 1
            with open('Database/url_construtores_faltantes.txt', 'a') as arq:
                arq.write(f"{construtor['url']} \n")

            continue

    print(f'pilotos sem imagens {construtores_sem_imagens}')

    return lista_construtores_url

In [139]:
lista_contrutores = obtem_img_url_todos_construtores(dados_construtores_tratados)

100%|██████████| 200/200 [03:57<00:00,  1.19s/it]

pilotos sem imagens 72


In [140]:
df_construtores_imagens = pd.DataFrame(lista_contrutores)

In [143]:
df_construtores_imagens.to_csv('Database/construtores_imagens.csv')